# **Project Name**    - **Retail Sales Pridiction**



##### **Project Type**    - Regression
##### **Contribution**    - Individual
##### **Team Member 1 -** - Syed Junaid Ali

# **Project Summary -**

**Rossmann Sales Dataset** - Rossmann Stores is a live dataset. During analysis of this dataset i observe that Rossmann dataset is a regression problem and our main goal is to predict the sales figures of Rossmann problem. In this Notebook i work on the following topics.

First of all i'll do some Exploratory Data Analysis to Analyse the Dataset. Analyse Trends and Seasonality in Rossmann dataset by Using Exponential Moving Averages.

Using following regeression models to predict the target:

1). Linear Regression

2). Lasso and Ridge Regression with cross validation(Grid search CV).

3). Random Forest Regression.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality and locality. With thousands of individual managers predicting sales based on their unique Li lances, the accuracy of results can be quite varied.

You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set.Note that some stores in the dataset were temporarily closed for refurbishment.

# ***Let's Begin !***

## ***Knowing our Dataset***

### Import Libraries

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score as r2, mean_squared_error as mse
import math
import itertools
from sklearn.ensemble import RandomForestRegressor

### Dataset Loading

In [ ]:
# Load Dataset
from google.colab import drive
drive.mount('/content/drive')
filepath = "/content/drive/MyDrive/regression datasets/"
Rossmann_store_df = pd.read_csv(filepath + 'Copy of Copy of Rossmann Stores Data.csv')
store_df = pd.read_csv(filepath + 'Copy of Copy of store.csv')

### Dataset First View

In [ ]:
# Dataset First Look
Rossmann_store_df.head()

In [ ]:
store_df.head()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
Rossmann_store_df.shape,store_df.shape

* Rossmann_store_df are having 1017209 rows and 9 columns.
* store_df are having 1115 rows and 10 columns.

### Dataset Information

In [ ]:
# Dataset Info
Rossmann_store_df.info()

In [ ]:
store_df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
len(Rossmann_store_df[Rossmann_store_df.duplicated()])

In [ ]:
len(store_df[store_df.duplicated()])

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
Rossmann_store_df.isnull().sum()

In [ ]:
null_columns = store_df[store_df.columns].isnull().sum()
null_columns

In [ ]:
null_columns = store_df[store_df.columns].isnull().sum()
percentage_missing_values = round(null_columns*100/store_df.count()[0],2)
percentage_missing_values

In [ ]:
# Visualizing the missing values
plt.figure(figsize=(10,5))
percentage_missing_values.plot(kind = 'bar')
plt.title("Percentage of Missing Values")

There are lots of null values found in this datataset.Having 3 missing values in **CompetitionDistance** , 345 missing values in **CompetitionOpenSinceMonth**, 345 missing values in **CompetitionOpenSinceYear**, 544 missing values in **Promo2SinceWeek**, 544 missing values in **Promo2SinceYear**, 544 missing values in **PromoInterval**. We can fill the null values further accordingly.

In [ ]:
new_store_df = store_df.copy()

#Replacing Nan values with mean.
mean_CompetitionDistance = new_store_df['CompetitionDistance'].mean()
new_store_df['CompetitionDistance'] = new_store_df['CompetitionDistance'].fillna(mean_CompetitionDistance)

In [ ]:
#Replacing Nan values with mode.
mode_CompetitionOpenSinceMonth = new_store_df['CompetitionOpenSinceMonth'].mode().iloc[0]
new_store_df['CompetitionOpenSinceMonth'] = new_store_df['CompetitionOpenSinceMonth'].fillna(mode_CompetitionOpenSinceMonth)

In [ ]:
CompetitionOpenSinceYear_mode = new_store_df['CompetitionOpenSinceYear'].mode().iloc[0]
new_store_df['CompetitionOpenSinceYear'] = new_store_df['CompetitionOpenSinceYear'].fillna(CompetitionOpenSinceYear_mode)

In [ ]:
Promo2SinceWeek_mode = new_store_df['Promo2SinceWeek'].mode().iloc[0]
new_store_df['Promo2SinceWeek'] = new_store_df['Promo2SinceWeek'].fillna(Promo2SinceWeek_mode)

In [ ]:
Promo2SinceYear_mode = new_store_df['Promo2SinceYear'].mode().iloc[0]
new_store_df['Promo2SinceYear'] = new_store_df['Promo2SinceYear'].fillna(Promo2SinceYear_mode)

In [ ]:
PromoInterval_mode = new_store_df['PromoInterval'].mode().iloc[0]
new_store_df['PromoInterval'] = new_store_df['PromoInterval'].fillna(PromoInterval_mode)
new_store_df.isnull().sum()

In [ ]:
#Transforming categorical features to numerical values.
new_store_df['StoreType'] = new_store_df['StoreType'].map({'a':0,'b':1,'c':2,'d':3})
new_store_df['Assortment'] = new_store_df['Assortment'].map({'a':1,'b':2,'c':3})
new_store_df['PromoInterval'] = new_store_df['PromoInterval'].map({'Jan,Apr,Jul,Oct' : 1,'Feb,May,Aug,Nov': 2,'Mar,Jun,Sept,Dec': 3})

In [ ]:
new_store_df

In [ ]:
#lets merge both datasets.
main_store_df  = pd.merge(Rossmann_store_df,new_store_df,how = 'left', on = 'Store')
main_store_df.head()

In [ ]:
main_store_df.shape

### Dataset Information

**Rossmann Stores Data.csv** - Previous data including target variable(sales)

**store.csv** - Having additional information about stores

**Features**

- Id - an Id that represents a (Store, Date) duple within the test set

- Store - a unique Id for each store(integer)

- Sales(Target variable) - the sales for any given day (this is what you are predicting for)(numeric)

- Customers - indicates the total number of customers on a given day(numeric)

- Open - indicates whether the store was open: 0 = closed, 1 = open(categorical)

- StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None(categorical)

- SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools(categoical)

- StoreType - having different types of  stores like a, b, c, d(categorical)

- Assortment - having different types of  assortment level: a = basic, b = extra, c = extended(categorical)

- CompetitionDistance - having distance in meters to the nearest competitor store(numeric)

- CompetitionOpenSince[Month/Year] - gives the approximate year and month in Which the nearest competitor was opened(numeric)

- Promo - indicates whether a store is running a promo on that day(categotical)

- Promo2 - indicates that store running consecutive promotions or not in which 0 = store is not participating, 1 = store is participating(categorical)

- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2(numeric)

- PromoInterval - describes the consecutive intervals when Promo2 is started, naming the months of the promotion is started a new. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store.(categorical)


## ***Understanding the Variables***

In [ ]:
# Dataset Columns
main_store_df.columns

In [ ]:
# Dataset Describe
main_store_df.describe().apply(lambda x: x.apply('{0:.2f}'.format))

In [ ]:
main_store_df.info()

In [ ]:
main_store_df['StateHoliday'].value_counts()

In [ ]:
#Transforming categorical features to numerical values.
main_store_df.loc[main_store_df['StateHoliday'] == '0', 'StateHoliday'] = 0
main_store_df.loc[main_store_df['StateHoliday'] == 'a', 'StateHoliday'] = 1
main_store_df.loc[main_store_df['StateHoliday'] == 'b', 'StateHoliday'] = 2
main_store_df.loc[main_store_df['StateHoliday'] == 'c', 'StateHoliday'] = 3
main_store_df['StateHoliday'] = main_store_df['StateHoliday'].astype(int, copy = False)
print('Unique -', main_store_df['StateHoliday'].unique(), '; Data Type -', main_store_df['StateHoliday'].dtype)

In [ ]:
#Transforming categorical features to numerical values.
main_store_df.loc[main_store_df['Assortment'] == 'a', 'Assortment'] = 1
main_store_df.loc[main_store_df['Assortment'] == 'b', 'Assortment'] = 2
main_store_df.loc[main_store_df['Assortment'] == 'c', 'Assortment'] = 3
main_store_df['Assortment'] = main_store_df['Assortment'].astype(int, copy = False)
print('Unique -', main_store_df['Assortment'].unique(), '; Data Type -', main_store_df['Assortment'].dtype)

In [ ]:
#Transforming categorical features to numerical values.
main_store_df.loc[main_store_df['Assortment'] == 'a', 'Assortment'] = 1
main_store_df.loc[main_store_df['Assortment'] == 'b', 'Assortment'] = 2
main_store_df.loc[main_store_df['Assortment'] == 'c', 'Assortment'] = 3
main_store_df.loc[main_store_df['Assortment'] == 'd', 'Assortment'] = 4
main_store_df['Assortment'] = main_store_df['Assortment'].astype(int, copy = False)
print('Unique -', main_store_df['Assortment'].unique(), '; Data Type -', main_store_df['Assortment'].dtype)


In [ ]:
# code for changing format of date from object to datetime
main_store_df['Date'] = pd.to_datetime(main_store_df['Date'], format= '%Y-%m-%d')

In [ ]:
main_store_df['CompetitionOpenSinceYear']= main_store_df['CompetitionOpenSinceYear'].astype(int)
main_store_df['Promo2SinceYear']= main_store_df['Promo2SinceYear'].astype(int)

In [ ]:
main_store_df['CompetitionOpenSinceMonth'] = pd.DatetimeIndex(main_store_df['Date']).month

In [ ]:
main_store_df['CompetitionDistance']= main_store_df['CompetitionDistance'].astype(int)
main_store_df['Promo2SinceWeek']= main_store_df['Promo2SinceWeek'].astype(int)

# **Exploratory Data Analysis**

In [ ]:
#Depending variable "Sales".
plt.figure(figsize=(15,10))
sns.distplot(main_store_df['Sales'],color="y")

In [ ]:
import math
plt.figure(figsize=(15,10))
sns.distplot(main_store_df['Sales'].apply(math.sqrt),color="y")

In [ ]:
# having sales(target variable) is 0 with store not open.
main_store_df[(main_store_df.Sales == 0) & (main_store_df.Open == 0)]

So, there are lots of rows which are having 0 value in Sales column because of store not open due to some reason.

In [ ]:
# numeric features
numeric_features = main_store_df.describe().columns

In [ ]:
# Distribution of numeric features.
for col in numeric_features[1:]:
  fig = plt.figure(figsize=(8, 5))
  ax = fig.gca()
  feature = main_store_df[col]
  feature.hist(bins=50, ax = ax)
  ax.axvline(feature.mean(), color='magenta', linestyle='dashed', linewidth=2)
  ax.axvline(feature.median(), color='cyan', linestyle='dashed', linewidth=2)
  ax.set_title(col)
plt.show()

By plotting above graphs we see that there are so many features which are not noramaly distributed.We can transform them later.

In [ ]:
main_store_df.columns

In [ ]:
trend_cols = ['CompetitionOpenSinceYear','Promo2SinceYear','CompetitionOpenSinceMonth','DayOfWeek']

## **Trend of sales during the period of time.**

In [ ]:
for col in trend_cols[:]:
  fig = plt.figure(figsize=(15, 6))
  ax = fig.gca()
  #feature = main_store_df[col]
  sns.pointplot(x= col, y= 'Sales', data=main_store_df)

  ax.set_title(col + ' vs Sales')
plt.show()

- **CompetitionOpenSinceYear vs Sales** - the above graph shows the relation between CompetitionOpenSinceYear and Sales that more sales are made in the year of 1900 when there is less competition, as year increases the new competitor stores comes into place which results in declined in sales.

- **Promo2SinceYear vs Sales** - the above graph shows the relation between Promo2SinceYear and Sales it effect the sales but intead of doing promo in the year of 2012 and 2013 sales will increased.

- **CompetitionOpenSinceMonth vs Sales** - the above graph shows the relation between CompetitionOpenSinceMonth and Sales that sales are high in the month of January, March, May, June, August in comparision to other months.

- **DayOfWeek vs sales** - the above graph shows the relation between DayOfWeek and Sales as the sales are high on Monday in comparision to other days of week and on Saturday and Sunday sales have decreased almost near to zero because some store are closed on these day.

**Barplot for categorical columns.**

In [ ]:
for col in ['StoreType','Assortment','Promo','StateHoliday','SchoolHoliday']:
  fig = plt.figure(figsize=(8, 6))
  ax = fig.gca()
  sns.barplot(x= col, y= 'Sales', data=main_store_df)
  ax.set_title(col + ' vs Sales')
plt.show()

- **StoreType vs Sales** - Above graph shows that store 1 has the maximum sales followed by other stores.
- **Assortment vs Sales** - As per the above graph we found that assortment 2 are having maximum sales in all stores.
- **Promo vs Sales**  = In Promo we have only 0 = not doing promos and 1 = doing promos,so we found the store who are continuously doing promos are have more sales than the stored who are not doing promos.
- **StateHoliday** - In this we three categories 0 = Public holiday, 1 = Easter holiday, 2 = Christmas, 3 = None we observe that stores are mainly closed on Public holiday's only.
- **SchoolHoliday** - We have 0 = stores closed on school holiday and 1 = stores open on school holiday as we can see above that store open on school holiday's are having more sales than normal.

In [ ]:
merged_df = pd.merge(Rossmann_store_df,store_df,how = 'left', on = 'Store')

In [ ]:
fig, axes = plt.subplots(2, 2,figsize=(15,10) )
palette = itertools.cycle(sns.color_palette(n_colors=4))
plt.subplots_adjust(hspace = 0.28)
axes[0,0].bar(merged_df.groupby(by="StoreType").count().Store.index ,merged_df.groupby(by="StoreType").count().Store)
axes[0,0].set_title("Number of Stores per Store Type \n Fig 1.1")
axes[0,1].bar(merged_df.groupby(by="StoreType").sum().Store.index,merged_df.groupby(by="StoreType").sum().Sales/1e9)
axes[0,1].set_title("Total Sales per Store Type \n Fig 1.2")
axes[1,0].bar(merged_df.groupby(by="StoreType").sum().Customers.index,merged_df.groupby(by="StoreType").sum().Customers/1e6)
axes[1,0].set_title("Total Number of Customers per Store Type (in Millions) \n Fig 1.3")
axes[1,1].bar(merged_df.groupby(by="StoreType").sum().Customers.index,merged_df.groupby(by="StoreType").Sales.mean())
axes[1,1].set_title("Average Sales per Store Type \n Fig 1.4")
plt.show()
print('Store type count \n', store_df['StoreType'].value_counts().sort_values())

- As per the above plots we observe that 'a' type of stores and customers are having maximum count followed by other store types and 'b' type of stores and customers are having least count.
- We also observe that b type of stores are having highest average sales per customer followed by the other stores.

**Count of Promo's per Store Type**

In [ ]:
#Number of Promos per Store Type
count_ = sns.countplot(x="StoreType", hue = "Promo",order = ['a','b','c','d'], data=merged_df,palette=sns.color_palette(n_colors=3)).set_title("Number of Promo per Store Type")
merged_df.groupby(by=["StoreType","Promo"]).Promo.count()

**Different types of Assortment with respect to store type**

a = common things
b = extra things
c = highest variety of products

In [ ]:
#Different types of Assortment count per Store Type
count_ = sns.countplot(x="StoreType", hue = "Assortment",order = ['a','b','c','d'], data=merged_df,palette=sns.color_palette(n_colors=3)).set_title("Different types of Assortment count per Store Type")
merged_df.groupby(by=["StoreType","Assortment"]).Assortment.count()

We found that the stores are mainly having 'a' and 'c' type of assortments are in high demand.

In [ ]:
store_df.isnull().sum()

**Clonclusions of EDA**

- There are two datasets - a) Rossmann.csv & b) Store.csv
- shape of Rossmann dataset = (1017209,8), shape of store dataset = (1115, 10)
- On looking on datasets we found lots of Nan values in Store dataset.
- replaced null values with suitable values. In CompetitionDistance column only
  3 Nan values are there. So we replaced it with mean.
- Also Having null values in CompetitionOpenSinceMonth,
  CompetitionOpenSinceYear,Promo2SinceWeek,Promo2SinceYear, so i replacedd it with mode.
- In both dataset 'Store' column is common. So i do inner join on the basis of column 'Store'.
- there are columns 'StateHoliday', 'SchoolHoliday' & 'Assortment' with object data types so i change these to int data type.

## ***Feature Engineering & Data Pre-processing***

In [ ]:
numeric_features

**Correlation**

In [ ]:
numeric_features_1 = [x for x in numeric_features if x not in ['Store','Sales','StoreType','Assssorment','PromoInterval']]
numeric_features_1

In [ ]:
for col in numeric_features_1[0:-1]:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    feature = main_store_df[col]
    label = main_store_df['Sales']
    corr_ = feature.corr(label)
    plt.scatter(x=feature, y=label)
    plt.xlabel(col)
    plt.ylabel('Sales')
    ax.set_title('Sales vs ' + col + '- correlation: ' + str(corr_))
    z = np.polyfit(main_store_df[col], main_store_df['Sales'], 1)
    y_hat = np.poly1d(z)(main_store_df[col])

    plt.plot(main_store_df[col], y_hat, "r--", lw=1)

plt.show()

In [ ]:
# Correlation
plt.figure(figsize=(18,8))
correlation = main_store_df.corr()
sns.heatmap(abs(correlation), annot=True, cmap='coolwarm')


**Multicolinearity**

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calculate_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [ ]:
calculate_vif(main_store_df[[x for x in main_store_df.describe().columns if x not in ['Sales']]])

Multicolinearity of CompetitionOpenSinceYear column is high so we decided to drop it.

In [ ]:
calculate_vif(main_store_df[[x for x in main_store_df.describe().columns if x not in ['Sales','CompetitionOpenSinceYear']]])

Multicolinearity of Promo2SinceYear column is high so we decided to drop it.

In [ ]:
calculate_vif(main_store_df[[x for x in main_store_df.describe().columns if x not in ['Sales','CompetitionOpenSinceYear','Promo2SinceYear']]])


Now the Variance Inflation Factor(VIF) is less that 10.It looks quite well.

**lets deal with dependent variable(Sales) having 0 value.**

In [ ]:
#checking the saled column where values are 0.
main_store_df[(main_store_df  .Open == 0) & (main_store_df.Sales == 0)].count()[0]

we have total **172817** values in sales column which is 0.It means type store is temporarily closed.So i decided to drop these rows.

In [ ]:
#dropping the values.
model_df = main_store_df.drop(main_store_df[(main_store_df  .Open == 0) & (main_store_df.Sales == 0)].index)

In [ ]:
#new shape
model_df.shape

In [ ]:
#changing the variable into dummies.
model_df = pd.get_dummies(model_df, columns=['PromoInterval'])

In [ ]:
model_df.head()

**Training Model(excluding Sales = 0)**

In [ ]:
# defining dependent variable
dependent_variables = 'Sales'

# defining independent variable
independent_variables = list(model_df.columns.drop(['CompetitionOpenSinceMonth','Promo2SinceYear','Date','Sales']))

In [ ]:
independent_variables

In [ ]:
from scipy.stats import zscore

In [ ]:
# Create the data of independent variables
X = model_df[independent_variables].values

# Create the data of dependent variable
Y = model_df[dependent_variables].values

In [ ]:
# splitting the dataset
X_train, X_test, Y_train, Y_test = train_test_split( X,Y , test_size = 0.2, random_state = 0)
print(X_train.shape)
print(X_test.shape)

**Linear Regression**

In [ ]:
reg = LinearRegression().fit(X_train, Y_train)

In [ ]:
reg.score(X_train, Y_train)

In [ ]:
reg.coef_

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
sales_avg = model_df[dependent_variables].mean()

In [ ]:
from sklearn.metrics import mean_squared_error
#Model Evaluation
MSE  = mean_squared_error(Y_test,y_pred) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE = RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :" ,RMPSE)

In [ ]:
from sklearn.metrics import r2_score
#Coefficient of determination
r2 = r2_score(Y_test, y_pred) #R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(Y_test,y_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))


In [ ]:
#plotting actual and predicting values
plt.figure(figsize=(6,6))
plt.scatter(Y_test, y_pred, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(y_pred), max(Y_test))
p2 = min(min(y_pred), min(Y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

**Lasso Regression**

In [ ]:
from sklearn.linear_model import Lasso
lasso  = Lasso(alpha=0.1 , max_iter= 3000)

lasso.fit(X_train, Y_train)

In [ ]:
lasso.score(X_train, Y_train)

In [ ]:
y_pred_l = lasso.predict(X_test)

In [ ]:
#Model Evaluation
MSE  = mean_squared_error(Y_test,y_pred_l) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE = RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :" ,RMPSE)

In [ ]:
#Coffiecient of dtermination
r2 = r2_score(Y_test,y_pred_l) # R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(Y_test,y_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
from sklearn.model_selection import GridSearchCV
###  Hyperparameter Tuning & Cross validation
lasso = Lasso()
parameters = {'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=3)
lasso_regressor.fit(X_train,Y_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,lasso_regressor.best_params_)
print("\nUsing ",lasso_regressor.best_params_, " the negative mean squared error is: ", lasso_regressor.best_score_)

In [ ]:
y_pred_lasso = lasso_regressor.predict(X_test)

In [ ]:
#model evaluation
MSE  = mean_squared_error(Y_test,y_pred_lasso) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE = RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :" ,RMPSE)

In [ ]:
#Coefficient of dtermiation
r2 = r2_score(Y_test,y_pred_lasso) # R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(Y_test,y_pred_lasso))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
#plotting actual and predicting values
plt.figure(figsize=(6,6))
plt.scatter(Y_test, y_pred_lasso, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(y_pred_lasso), max(Y_test))
p2 = min(min(y_pred_lasso), min(Y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

**Ridge regression**

In [ ]:
from sklearn.linear_model import Ridge

ridge  = Ridge(alpha=0.1)

In [ ]:
ridge.fit(X_train,Y_train)

In [ ]:
ridge.score(X_train,Y_train)

In [ ]:
y_pred_ridge = ridge.predict(X_test)

In [ ]:
#Model Evaluation
MSE  = mean_squared_error(Y_test,y_pred_ridge) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE = RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :" ,RMPSE)

In [ ]:
#Coefficient of determination
r2 = r2_score(Y_test,y_pred_ridge) # R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(Y_test,y_pred_ridge))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
# Hyperprarameter tuning & Cross validation
ridge = Ridge()
parameters = {'alpha': [1e-15,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1,5,10,20,30,40,45,50,55,60,100]}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=3)
ridge_regressor.fit(X_train,Y_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,ridge_regressor.best_params_)
print("\nUsing ",ridge_regressor.best_params_, " the negative mean squared error is: ", ridge_regressor.best_score_)

In [ ]:
#Model Prediction
y_pred_r = ridge_regressor.predict(X_test)

In [ ]:
#Model Evaluation.
MSE  = mean_squared_error(Y_test,y_pred_r) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE = RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :" ,RMPSE)

In [ ]:
#Coefficient of determination
r2 = r2_score(Y_test,y_pred_r)
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(Y_test,y_pred_r))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
#plotting actual and predicting values
plt.figure(figsize=(6,6))
plt.scatter(Y_test, y_pred_r, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(y_pred_r), max(Y_test))
p2 = min(min(y_pred_r), min(Y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

**Decision Tree**

In [ ]:
decision_tree=DecisionTreeRegressor(max_depth=5)
decision_tree.fit(X_train,Y_train)

In [ ]:
y_pred_dt = decision_tree.predict(X_test)

In [ ]:
y_train_dt = decision_tree.predict(X_train)

In [ ]:
#Model Evaluation.
MSE  = mean_squared_error(Y_test,y_pred_dt) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE = RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :" ,RMPSE)

In [ ]:
#Coefficient of determination
r2 = r2_score(Y_test,y_pred_dt) #R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(Y_test,y_pred_dt))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
#plotting actual and predicting values
plt.figure(figsize=(6,6))
plt.scatter(Y_test, y_pred_dt, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(y_pred_dt), max(Y_test))
p2 = min(min(y_pred_dt), min(Y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

# **Model 2 (including all the observations of Sales)**

In [ ]:
#reviewing the dataset
main_store_df

Now first of all we have make dummies of '**PromoInteval**' column.

In [ ]:
main_store_df =  pd.get_dummies(main_store_df, columns=['PromoInterval'])

In [ ]:
main_store_df.head()

In [ ]:
main_store_df.shape

In [ ]:
#lets define dependent and independent variables and convert them into arrays
# defining dependent variable
dep_var = 'Sales'

# defining independent variable
indep_var = main_store_df.columns.drop(['Store', 'Promo2SinceYear','Date','Sales'])

In [ ]:
u = main_store_df[indep_var].values
v = main_store_df[dep_var].values

In [ ]:
main_store_df[indep_var]

Dooing train-test split considering test size 0.30.

In [ ]:
# splitting the dataset
u_train, u_test, v_train, v_test = train_test_split(u, v, test_size=0.30, random_state = 0)
print(u_train.shape)
print(u_test.shape)

**Linear Regression**

In [ ]:
# scaling the u values
scaler=StandardScaler()

u_train = scaler.fit_transform(u_train)
u_test = scaler.transform(u_test)

In [ ]:
# fitting the data into Lineat Regression Model
linear_reg = LinearRegression()
linear_reg.fit(u_train, v_train)

In [ ]:
#predicting now
v_pred=linear_reg.predict(u_test)
v_pred

In [ ]:
linear_reg.score(u_train,v_train)

In [ ]:
linear_reg.score(u_test,v_test)


In [ ]:
reg_Dataframe = pd.DataFrame(zip(v_test, v_pred), columns = ['actual', 'pred'])
reg_Dataframe

In [ ]:
#model evaluation
MSE  = mean_squared_error(v_test, v_pred) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE=RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :",RMPSE)

r2 = r2_score(v_test, v_pred) #R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(v_test,v_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
#plotting actual and predicting values
plt.figure(figsize=(6,6))
plt.scatter(v_test, v_pred, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(v_pred), max(v_test))
p2 = min(min(v_pred), min(v_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

**Lasso Regression**

In [ ]:
lasso = Lasso(alpha=1.0)

In [ ]:
lasso.fit(u_train, v_train)

In [ ]:
v_pred_lasso = lasso.predict(u_test) #prediction

In [ ]:
lasso.score(u_train, v_train) #lasso score

In [ ]:
pd.DataFrame(zip(v_test, v_pred_lasso), columns = ['actual', 'pred'])

In [ ]:
#model evaluation
MSE  = mean_squared_error(v_test, v_pred_lasso) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE=RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :",RMPSE)

r2 = r2_score(v_test, v_pred_lasso) #R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(v_test,v_pred_lasso))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
### Hyperparameter tuning & Cross validation
lasso = Lasso()
parameters = {'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=3)
lasso_regressor.fit(u_train,v_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,lasso_regressor.best_params_)
print("\nUsing ",lasso_regressor.best_params_, " the negative mean squared error is: ", lasso_regressor.best_score_)

In [ ]:
v_pred_lass = lasso_regressor.predict(u_test) # prediction

In [ ]:
#model evaluation
MSE  = mean_squared_error(v_test, v_pred_lass) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE=RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :",RMPSE)

r2 = r2_score(v_test, v_pred_lass) #R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(v_test,v_pred_lass))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
#plotting actual and predicting values
plt.figure(figsize=(6,6))
plt.scatter(v_test, v_pred_lass, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(v_pred_lass), max(v_test))
p2 = min(min(v_pred_lass), min(v_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

**Ridge Rergression**

In [ ]:
ridge = Ridge(alpha=0.5)

In [ ]:
ridge.fit(u_train, v_train)

In [ ]:
v_pred_rid=ridge.predict(u_test)

In [ ]:
ridge.score(u_test, v_test)

In [ ]:
pd.DataFrame(zip(v_test, v_pred_rid), columns = ['actual', 'pred'])

In [ ]:
#model evaluation
MSE  = mean_squared_error(v_test, v_pred_rid) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE=RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :",RMPSE)

r2 = r2_score(v_test, v_pred_rid) #R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(v_test,v_pred_rid))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
### Hyperparameter tuning & Cross validation
ridge2 = Ridge()
parameters = {'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100]}
ridge_regressor = GridSearchCV(ridge2, parameters, scoring='neg_mean_squared_error', cv=3)
ridge_regressor.fit(u_train,v_train)

In [ ]:
print("The best fit alpha value is found out to be :" ,ridge_regressor.best_params_)
print("\nUsing ",ridge_regressor.best_params_, " the negative mean squared error is: ", ridge_regressor.best_score_)

In [ ]:
v_pred_ridg = ridge_regressor.predict(u_test) # prediction

In [ ]:
#model evaluation
MSE  = mean_squared_error(v_test, v_pred_ridg) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE=RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :",RMPSE)

r2 = r2_score(v_test, v_pred_ridg) #R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(v_test,v_pred_ridg))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
#plotting actual and predicting values
plt.figure(figsize=(6,6))
plt.scatter(v_test, v_pred_ridg, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(v_pred_ridg), max(v_test))
p2 = min(min(v_pred_ridg), min(v_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

**Decision Tree**

In [ ]:
decision_tree=DecisionTreeRegressor(max_depth=5)
decision_tree.fit(u_train, v_train) #fitting model

In [ ]:
#predicting
v_pred_dt = decision_tree.predict(u_test)
v_train_dt = decision_tree.predict(u_train)

In [ ]:
#model evaluation
MSE  = mean_squared_error(v_test, v_pred_dt) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE=RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :",RMPSE)

r2 = r2_score(v_test, v_pred_dt) #R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(v_test,v_pred_dt))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
decisiontree_Dataframe = pd.DataFrame(zip(v_test, v_pred_dt), columns = ['actual', 'pred'])
decisiontree_Dataframe

In [ ]:
#plotting actual and predicted values
plt.figure(figsize=(5,5))
plt.scatter(v_test, v_pred_dt, c='crimson')
plt.yscale('log')
plt.xscale('log')

p3 = max(max(v_pred_dt), max(v_test))
p4 = min(min(v_pred_dt), min(v_test))
plt.plot([p3, p4], [p3, p4], 'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

**Random Forest**

In [ ]:
random_forest1=RandomForestRegressor(n_estimators =500,max_depth=8)
random_forest1.fit(u_train, v_train)

In [ ]:
v_pred_rf=random_forest1.predict(u_test)

In [ ]:
#model evaluation
MSE  = mean_squared_error(v_test, v_pred_rf) #Mean Squared Error
print("MSE :" , MSE)

RMSE = np.sqrt(MSE) #Root Mean Squared Error
print("RMSE :" ,RMSE)

RMPSE=RMSE/sales_avg #Root Mean Per Squared Error
print("RMPSE :",RMPSE)

r2 = r2_score(v_test, v_pred_rf) #R Squared & Adjusted R Squared
print("R2 :" ,r2)
print("Adjusted R2 : ",1-(1-r2_score(v_test,v_pred_rf))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

In [ ]:
#showing actual and predicted values
rf_Dataframe = pd.DataFrame(zip(v_test, v_pred_rf), columns = ['actual', 'pred'])
rf_Dataframe

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(v_test, v_pred_rf, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(v_pred_rf), max(v_test))
p2 = min(min(v_pred_rf), min(v_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Actual Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

## ***Future Work***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File
import joblib

joblib.dump(random_forest1, 'regression_model.joblib')

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Loading the File and predicting unseen data.
regg = joblib.load('regression_model.joblib')
predictions = regg.predict(u_test)
predictions

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion from Model Training**

We found that Sales column contains 172817 rows having 0 sale. So, i decided to create a new dataframe in which i removed rows having 0 values and tried to train our model. I used various algorithms such as Linear regression, Decision tree, Random Forest, Lasso & Ridge Regression with hyperparameter tuning and got accuracy score around 74%.

I also want to see the model accuracy on total dataset(including Sales = 0 rows). So i trained another model using the same algorithms and i got accuracy near about 92% which is far better than previous model.

So i came across with conclusion that removing sales rows having  0 values actually removes lot of information from dataset as it has 172817 rows which is quite large and therefore i decided not to remove those values.i got my best rmpse score from Random Forest model,i tried taking an optimum parameter so that our model doesnt overfit.

# **Conclusion from EDA**

- By plotting Sales vs competition Open Since Month shows sales go increasing from November and highest in the month of December.
- By plotting Sales vs day of week, sales are highest on Monday and start declining from Tuesday to Saturday and on Sunday sales are almost near to Zero.
- By Plotting Promotion vs Sales indicates that promotion helps in increasing Sales.
- Types of Store plays an important role in opening pattern of stores.
- Type ‘b’ stores never closed except for refurbishment or any other reason.
- Type ‘b’ stores have comparatively having higher average sales per customer and it is mostly constant with peaks appears on weekends.
- Assortment ‘b’ is only offered by Store Type ‘b’.
- I can observe that most of the stores remain closed during State Holidays. But it is interesting to note that the number of stores opened during School Holidays were more than that stores which are opened on State Holidays.





### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***